### import libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchsummary
import preprocessing
import cnn

### set device

In [2]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### set hyperparameters

In [3]:
#in_channels = 1    # 28x28 = 784, size of MNIST images (grayscale)
#hidden_size = 100
num_classes = 24
learning_rate = 0.001
batch_size = 64
num_epochs = 20

### load & preprocess data

In [4]:
my_transforms = torchvision.transforms.Compose([
    preprocessing.ToTensor(), 
])

dataset = preprocessing.SignDataset(csv_path="data/sign_mnist_train.csv", transform=my_transforms)
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

dataset_test = preprocessing.SignDataset(csv_path="data/sign_mnist_test.csv", transform=my_transforms)
dataloader_test = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

### data summary

In [5]:
print('type(dataloader):', type(dataloader))
print('----------------------------------')
print('data_iter = iter(dataloader)')
data_iter = iter(dataloader)
print('----------------------------------')
print('single_batch = data_iter.next()')
#single_batch = data_iter.next()
#print('type(single_batch) :', type(single_batch))
#print('len(single_batch)  :', len(single_batch))
print('----------------------------------')
print('images, labels = single_batch')
#images, labels = single_batch
#print('images.shape:', images.shape)
#print('labels.shape:', labels.shape)
#print(type(labels[0]))

type(dataloader): <class 'torch.utils.data.dataloader.DataLoader'>
----------------------------------
data_iter = iter(dataloader)
----------------------------------
single_batch = data_iter.next()
----------------------------------
images, labels = single_batch


### Initialize network

In [6]:
# Create the network and look at it's text representation
net = cnn.ConvNN().to(device)
torchsummary.summary(net, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
         MaxPool2d-2           [-1, 10, 13, 13]               0
            Conv2d-3           [-1, 20, 11, 11]           1,820
         MaxPool2d-4             [-1, 20, 5, 5]               0
            Conv2d-5             [-1, 30, 3, 3]           5,430
         Dropout2d-6             [-1, 30, 3, 3]               0
            Linear-7                  [-1, 270]          73,170
            Linear-8                   [-1, 24]           6,504
        LogSoftmax-9                   [-1, 24]               0
Total params: 87,024
Trainable params: 87,024
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 0.33
Estimated Total Size (MB): 0.43
---------------------------------------------

### train the model

In [7]:
import numpy as np
a = np.float32(1)
print('a:', a)
print(type(a))
b = np.array(a)
print('b:', b)
print(type(b))
c = torch.from_numpy(b)
print('c:', c)
print(type(c))

d = torch.from_numpy(b).type(torch.LongTensor)
print('d:', d)
print('type(d):', type(d))
print('d.shape:', d.shape)

a: 1.0
<class 'numpy.float32'>
b: 1.0
<class 'numpy.ndarray'>
c: tensor(1.)
<class 'torch.Tensor'>
d: tensor(1)
type(d): <class 'torch.Tensor'>
d.shape: torch.Size([])


In [8]:
trained_model = cnn.train(
    loader=dataloader, 
    model=net, 
    num_epochs=2, 
    learning_rate=learning_rate, 
    device=device
    )

epoch [1/2], batch [200/429], loss = 1.8517
epoch [1/2], batch [400/429], loss = 1.7552
epoch [2/2], batch [200/429], loss = 0.9650
epoch [2/2], batch [400/429], loss = 0.8346


### model performance

In [9]:
cnn.check_accuracy(loader=dataloader, model=trained_model,device=device)
cnn.check_accuracy(loader=dataloader_test, model=trained_model, device=device)

Got 22554 / 27455 with accuracy 82.15 %
Got 4828 / 7172 with accuracy 67.32 %
